In [1]:
from swda_time import CorpusReader
from random import randrange
from itertools import chain
from complexity import ComplexityMeasures
from nltk.tree import Tree
import numpy as np

swbd = CorpusReader('swda_complete', 'swda_complete/swda-metadata.csv')
bnc = CorpusReader('bnc_complete', 'bnc_complete/bnc-metadata.csv')

corpus = swbd
complexity = ComplexityMeasures(corpus)

In [2]:
#find a random example and compute the non-normalized measures
iterator = corpus.iter_utterances()

for i in range(randrange(1000)):
    s = iterator.next()

for tree in s.trees:
    print tree
    print 'Length: {}'.format(complexity.length(tree))
    print 'Depth: {}'.format(complexity.depth(tree))
    print 'Width: {}'.format(complexity.width(tree))
    #print 'Depth*width: {}'.format(complexity.balanced(tree))
    print 'Balanced depth*width: {}'.format(complexity.balanced2(tree))
    print 'Average depth: {}'.format(complexity.avdepth(tree))
print "Lu's measures:"
for item in complexity.lus_measures(s.trees).items():
    print'{}:{}'.format(*item)
    

utterance 885

(S
  (EDITED
    (RM (-DFL- \[))
    (EDITED (RM (-DFL- \[)) (NP-SBJ (DT That)) (, ,) (IP (-DFL- \+)))
    (S
      (NP-SBJ (DT that))
      (RS (-DFL- \]))
      (VP (MD would) (VP-UNF (VB be))))
    (, ,)
    (IP (-DFL- \+)))
  (NP-SBJ (DT That))
  (VP
    (MD would)
    (VP
      (VB be)
      (RS (-DFL- \]))
      (ADJP-PRD (RB culturally) (JJ shocking))))
  (. .)
  (-DFL- E_S))
Length: 18
Depth: 7
Width: 1.48648648649
Balanced depth*width: 1.48012718601
Average depth: 5.21052631579
Lu's measures:
MLC:10.5
C/S:2.0
CP/C:0.0
C/T:2.0
DC/C:0.0
T/S:1.0
DC/T:0.0
MLT:18.0
CN/C:0.0
CT/T:0.0
VP/T:2.0
CP/T:0.0
CN/T:0.0


In [3]:
#print the normalized measures for the random example
for tree in s.trees:
    print 'Normalized depth: {}'.format(complexity.ndepth(tree))
    print 'Normalized width: {}'.format(complexity.nwidth(tree))
    #print 'Normalized depth*width: {}'.format(complexity.nbalanced(tree))
    print 'Normalized balanced depth*width: {}'.format(complexity.nbalanced2(tree))
    print 'Normalized average depth: {}'.format(complexity.n_avdepth(tree))

Normalized depth: 0.688282538738
Normalized width: 0.989703030696
Normalized balanced depth*width: 0.836814067442
Normalized average depth: 0.795676392573


In [ ]:
# dicts with the measures for easy iteration
# add additional measures here
measures = {'length': complexity.length,
            'depth': complexity.depth,
            'width': complexity.width,
            'depth*width': complexity.balanced,
            'balanced depth*width': complexity.balanced2,
            'average depth': complexity.avdepth,
            'normalized depth': complexity.ndepth,
            'normalized width': complexity.nwidth,
            'normalized depth*width': complexity.nbalanced,
            'normalized average depth': complexity.n_avdepth,
            'normalized balanced depth*width': complexity.nbalanced2}

non_normalized_measures = {'length': complexity.length,
                           'depth': complexity.depth,
                           'width': complexity.width,
                           'average depth': complexity.avdepth,
                           'depth*width': complexity.balanced,
                           'balanced depth*width': complexity.balanced2}

normalized_measures = {'normalized depth': complexity.ndepth,
                       'normalized width': complexity.nwidth,
                       'normalized depth*width': complexity.nbalanced,
                       'normalized average depth': complexity.n_avdepth,
                       'normalized balanced depth*width': complexity.nbalanced2}

# dict of sets (to avoid duplicates) for the results
ordered_results = dict([(measure, []) for measure in measures])

for conv in corpus.iter_transcripts(display_progress=False):
    for utt in conv.utterances:
        for tree in utt.trees:
            #Trees need to be cast to string to be hashable for duplicate removal
            string = str(tree)
            for measure in measures:
                ordered_results[measure].append((conv.conversation_no, measures[measure](tree), string))

results = {}
#remove duplicates from results            
for measure in ordered_results:
    results[measure] = list(set(ordered_results[measure]))
    results[measure].sort(key=lambda x: x[1])

#compute results closest to average
av_results = {}

for measure in non_normalized_measures:
    mean = np.mean([x[1] for x in results[measure]])
    av_results[measure] = sorted(results[measure], key=lambda x: abs(x[1] - mean), reverse=True)
for measure in normalized_measures:
    av_results[measure] = sorted(results[measure], key=lambda x: abs(x[1] - 1), reverse=True)

In [10]:
# code to display top, bot and closest to average n trees per measure

n = 5 #length of lists to be displayed

# modifiers with associated functions
mods = {'Most': lambda x: x[len(x)-n:], 'Least': lambda x: x[:n]}

for measure in measures:
    for modifier in mods:
        print '{} complex trees by {}:\n'.format(modifier, measure)
        for i, item in enumerate(mods[modifier](results[measure]), 1):
            tree = Tree.fromstring(item[2])
            text = str(tree.flatten()).replace('\n', '')
            print '{}.(from conv. no. {}) {} \n {}\n'.format(i, item[0], text, str(tree))
        print '\n'

for measure in av_results:
    print 'Closest to average trees by {}\n'.format(measure)
    for i, item in enumerate(mods['Most'](av_results[measure]), 1):
        tree = Tree.fromstring(item[2])
        text = str(tree.flatten()).replace('\n', '')
        print '{}.(from conv. no. {}) {} \n {}\n'.format(i, item[0], text, tree)

Most complex trees by MLC:



KeyError: 'MLC'

In [6]:
for measure in results:
    print len(results[measure])
    
for i, item in enumerate(ordered_results['normalized depth']):
    assert item[2] == ordered_results['normalized average depth'][i][2]

156967
156967
156967
156967
156967
156967
156967
156967
156967
156967
156967


In [7]:
# code to find distinguishing sentences for each (sensible) pair of normalized measures

i = 0
measure_list = list(normalized_measures)
diff = {}

while i < len(measure_list):
    one = measure_list[i]
    data_one = [x[1] for x in ordered_results[one]]
    for other in measure_list[i + 1:]:
        data_other = [x[1] for x in ordered_results[other]]
        comp = zip(data_one, data_other)
        index, values = max(enumerate(comp), key=lambda x: abs(x[1][0] - x[1][1]))
        # sanity check: Are these actually values for the same tree?
        #               Do they actually have these values?
        assert ordered_results[one][index][2] == ordered_results[other][index][2]
        assert ordered_results[one][index][1] == values[0]
        assert ordered_results[other][index][1] == values[1]
        tree = ordered_results[one][index][2]
        diff[(one, other)] = (tree, values)
    i += 1
    
for pair in diff:
    print 'Largest difference in value between {} and {}:'.format(*pair)
    tree, values = diff[pair]
    print 'Value for {pair[0]}: {values[0]}, value for {pair[1]}: {values[1]}'.format(pair=pair, values=values)
    print Tree.fromstring(tree)

Largest difference in value between normalized depth and normalized average depth:
Value for normalized depth: 2.24009090301, value for normalized average depth: 1.51915636241
(ROOT (FRAG (ADJP (RB Exactly) (SBAR (S (VP (VBZ happens))))) (. .)))
Largest difference in value between normalized depth*width and normalized balanced depth*width:
Value for normalized depth*width: 1.04799347071, value for normalized balanced depth*width: 2.28571428571
(S
  (EDITED
    (RM (-DFL- \[))
    (S (NP-SBJ (PRP it)) (VP-UNF (BES 's)))
    (, ,)
    (IP (-DFL- \+)))
  (NP-SBJ (NP (PRP it)) (SBAR (-NONE- *ICH*-5)))
  (VP
    (BES 's)
    (RS (-DFL- \]))
    (ADJP-PRD (JJ hard))
    (EDITED
      (RM (-DFL- \[))
      (S (NP-SBJ (-NONE- *)) (VP-UNF (TO to)))
      (, ,)
      (IP (-DFL- \+)))
    (PRN (S (NP-SBJ (PRP I)) (VP (VBP guess))) (, ,))
    (SBAR-5
      (IN for)
      (S
        (NP-SBJ (PRP me))
        (VP
          (TO to)
          (RS (-DFL- \]))
          (VP
            (VB justify)
    

In [8]:
all_swbd_trees = {}

for conv in swbd.iter_transcripts(display_progress=False):
    trees = {'A': set(), 'B': set()}
    for ut in conv.utterances:
        trees[ut.caller].add(tuple([str(tree) for tree in ut.trees]))
    all_swbd_trees[conv.conversation_no] = trees

In [9]:
# code to check whether there are parts of either corpus for which Lu's measures make sense
# in the sense of being non-zero for more than half of each speaker's utterances that have distinct trees

lu_measure_list = ['MLC', 'MLT',
                   'C/S',
                   'C/T', 'CT/T', 'DC/C', 'DC/T',
                   'CP/C', 'CP/T', 'T/S',
                   'CN/C', 'CN/T', 'VP/T']
    
swbd_results = dict([(conv, dict([(measure, dict([(speaker, []) for speaker in ('A', 'B')])) for measure in lu_measure_list])) for conv in all_swbd_trees])
swbd_defined = dict([(conv, dict([(measure, dict([(speaker, []) for speaker in ('A', 'B')])) for measure in lu_measure_list])) for conv in all_swbd_trees])
  
good_convs = dict([(measure, []) for measure in lu_measure_list])
                    
for conv in all_swbd_trees:
    for speaker in all_swbd_trees[conv]:
        for trees in all_swbd_trees[conv][speaker]:
            Trees = [Tree.fromstring(tree) for tree in trees]
            measures = complexity.lus_measures(Trees)
            for measure in measures:
                swbd_results[conv][measure][speaker].append(measures[measure])
                if measures[measure] == 0.0 or measures[measure] == None:
                    swbd_defined[conv][measure][speaker].append(0.0)
                else:
                    swbd_defined[conv][measure][speaker].append(1.0)
        for measure in swbd_defined[conv]:
            swbd_defined[conv][measure][speaker] = np.mean(swbd_defined[conv][measure][speaker])
    for measure in good_convs:
        if swbd_defined[conv][measure]['A'] >= 0.5 and swbd_defined[conv][measure]['B'] >= 0.5:
            good_convs[measure].append(conv)

for measure, convs in good_convs.items():
    print '{}: {}'.format(measure, len(convs))
                    

                    
"""
all_bnc_trees = {}    
    
for conv in bnc.iter_transcripts(display_progess=False):
    trees = {'A': set(), 'B': set()}
    for ut in conv.utterances:
        trees[ut.caller].add(tuple([str(tree) for tree in ut.trees]))
    all_bnc_trees[conv.conversation_no] = trees
"""

MLC: 1091
C/S: 1091
CP/C: 0
C/T: 1075
DC/C: 1
T/S: 1123
DC/T: 1
MLT: 1123
CN/C: 30
CT/T: 1
VP/T: 1095
CP/T: 0
CN/T: 35


"\nall_bnc_trees = {}    \n    \nfor conv in bnc.iter_transcripts(display_progess=False):\n    trees = {'A': set(), 'B': set()}\n    for ut in conv.utterances:\n        trees[ut.caller].add(tuple([str(tree) for tree in ut.trees]))\n    all_bnc_trees[conv.conversation_no] = trees\n"